# Web Scraping mtggoldfish.com

So, yes, I'm a total nerd. I like Magic the Gathering. But at least I don't LARP!!!

Let's try scraping mtggoldfish.com, which is an amazing site with all kinds of data and news on the game. But particularly, it has prices of cards, which we could (in theory) scrape, then use to build a ML model to predict card prices, AND THEN USE IT TO BECOME FABULOSULY WEALTHY AND TAKE OVER THE WORLD.

Or maybe just get some Commander singles at a decent price. WIZARDS PLEASE JUST REPRINT SOME DECENT LANDS SO I DON'T HAVE TO RUN ALL TAPLANDS FOR MY 5 COLOR JANK!!!

# Setup

In [1]:
import re
import requests
from bs4 import BeautifulSoup

In [2]:
url = r'https://www.mtggoldfish.com/'

r = requests.get(url)

data = r.text

So here's what the raw html looks like as a text file. Pretty much unintelligable.

In [4]:
data[:10000]

'<!DOCTYPE html>\n<html lang=\'en\' xmlns=\'http://www.w3.org/1999/xhtml\'>\n<head>\n<title>MTGGoldfish - Magic the Gathering Prices, Decks and Strategy</title>\n<meta name="description" content="Magic the Gathering and Magic Online prices, decks and strategy.">\n<meta name="keywords" content="magic: the gathering price list, mtg price list, mtgo price list, decks, metagame, archetype, standard, modern, legacy, graphs, charts, mtgo prices, prices, speculation, speculators, trends, stocks">\n<meta property="og:title" content="MTGGoldfish - Magic the Gathering Prices, Decks and Strategy">\n<meta property="og:type" content="website">\n<meta property="og:url" content="https://www.mtggoldfish.com/">\n<meta property="og:description" content="Magic the Gathering and Magic Online prices, decks and strategy.">\n<meta name="twitter:card" content="summary">\n<meta name="twitter:site" content="@mtggoldfish">\n<link rel="alternate" type="application/atom+xml" title="ATOM" href="https://www.mtggoldf

But, we can use beautiful soup to get some nice data out of this doc.

In [5]:
soup = BeautifulSoup(data)

So, step 1 is to just take a look at what we're dealing with. BS (beautiful soup) has a nice command so see all the html with proper indents, which makes it more readable.

In [9]:
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   MTGGoldfish - Magic the Gathering Prices, Decks and Strategy
  </title>
  <meta content="Magic the Gathering and Magic Online prices, decks and strategy." name="description"/>
  <meta content="magic: the gathering price list, mtg price list, mtgo price list, decks, metagame, archetype, standard, modern, legacy, graphs, charts, mtgo prices, prices, speculation, speculators, trends, stocks" name="keywords"/>
  <meta content="MTGGoldfish - Magic the Gathering Prices, Decks and Strategy" property="og:title"/>
  <meta content="website" property="og:type"/>
  <meta content="https://www.mtggoldfish.com/" property="og:url"/>
  <meta content="Magic the Gathering and Magic Online prices, decks and strategy." property="og:description"/>
  <meta content="summary" name="twitter:card"/>
  <meta content="@mtggoldfish" name="twitter:site"/>
  <link href="https://www.mtggoldfish.com/feed" rel="alternate" title="A

But, this is still pretty hard to parse. Let's get rid of all the html tags and just look at the text.

In [16]:
text = soup.get_text(strip = True)
text

'MTGGoldfish - Magic the Gathering Prices, Decks and Strategy(function() {\nvar _fbq = window._fbq || (window._fbq = []);\nif (!_fbq.loaded) {\nvar fbds = document.createElement(\'script\');\nfbds.async = true;\nfbds.src = \'//connect.facebook.net/en_US/fbds.js\';\nvar s = document.getElementsByTagName(\'script\')[0];\ns.parentNode.insertBefore(fbds, s);\n_fbq.loaded = true;\n}\n_fbq.push([\'addPixelId\', \'928681210525176\']);\n})();\nwindow._fbq = window._fbq || [];\nwindow._fbq.push([\'track\', \'PixelInitialized\', {}]);window["nitroAds"] = window["nitroAds"] || {\n    createAd: function() {\n      window.nitroAds.queue.push(["createAd", arguments]);\n    },\n    queue: []\n  };Toggle navigationPricesPrice ListsMovers and ShakersDecksMetagameFormat StaplesSuperBrewUser Submitted DecksUser Submitted DecksSubmit a DeckArticlesToolsMy CollectionMy DecksMy Price AlertsPremium MembershipDeck PricerChrome Browser ExtensionShopUpcoming ReleasesTheros: Beyond DeathJan 24Recent ReleasesSecr

In [14]:
links = soup.find_all('a')

display(f'There are {len(links)} links')

for link in links[:10]:
    print(link)

'There are 162 links'

<a href="/"><img alt="MTGGoldfish Logo" class="layout-header-banner-logo-img sprite-logo-with-type-large" src="https://assets1.mtggoldfish.com/assets/s-d69cbc552cfe8de4931deb191dd349a881ff4448ed3251571e0bacd0257519b1.gif"/>
</a>
<a class="navbar-brand layout-navbar-brand" href="/">
<img alt="MTGGoldfish Icon" class="layout-brand-image img-circle sprite-goldfish-navbar-logo" src="https://assets1.mtggoldfish.com/assets/s-d69cbc552cfe8de4931deb191dd349a881ff4448ed3251571e0bacd0257519b1.gif"/>
<!-- = image_tag("logo-with-type-large.png") -->
<!-- = image_tag("logo-with-type.png") -->
</a>
<a class="dropdown-toggle" data-toggle="dropdown" href="#">
Prices
<span class="caret"></span>
</a>
<a href="/prices/standard">Price Lists</a>
<a href="/movers/standard">Movers and Shakers</a>
<a class="dropdown-toggle" data-toggle="dropdown" href="#">
Decks
<span class="caret"></span>
</a>
<a href="/metagame/standard">Metagame</a>
<a href="/format-staples/standard">Format Staples</a>
<a href="/superbrew"

Wow, that's a lot of links! Well, we could in theory go to each of those links, then go to each link that each of those links goes to (if we haven't been there already), and so on. This would allow us to trawl the whole site.